In [ ]:
""" Create a car maze sort of simulation genetic genetic algorithm """

'''

Pre-sets:
Inputs:
    ray casts in front, and two sides
Genes:
    layers of neural net represent each "gene"
        neural network weights 
    size: 3 inputs, 1 hidden, 3 outputs(left, right, side)


Inital:
    create multiple parents with random genes(weights)

Children:
mutate:
randomly multiply weights by num

Crossover:
choose two parents
randomly choose weights and biases to add

Fitness:
which car hits the most rewards

Choose best:
top two best of generation 


NEXT STEPS:

create population that last for certain time period:
- add to fitnesses when hit prizes
- stop movement when collision
- keep running timer




'''


import pygame, sys
import random
import math
from copy import deepcopy
import datetime
import time

pygame.init()
WIDTH, HEIGHT = 600, 700
DISPLAYSURF = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()


class Chromosome:
    def __init__(self, genes, strategy):
        self.Genes = genes
        self.Age = 0
        self.Strategy = strategy
        self.is_best = False
        self.Fitness = 0
    
def mutate(parent_network, wt_mutate_chance=0.05):
    child_genes = deepcopy(parent_network)
    for i, layer in enumerate(child_genes):
        child_genes[i].mutate(wt_mutate_chance)
    
    return child_genes
            
def crossover(p1_network, p2_network):
    child_genes = deepcopy(p1_network)
    for i in range(len(p1_network)):
        for j in range(len(p1_network[i].Neurons)):
            child_genes[i].Neurons[j] = random.choice([p1_network[i].Neurons[j], p2_network[i].Neurons[j]])
    return child_genes    
# number of weights per neuron = number of inputs    

def feedforward(network, inp):
    curr_out = inp[:]
    for i, layer in enumerate(network):
        curr_out = layer.get_outputs(curr_out)
    return curr_out
    

    
    
class Layer:
    def __init__(self, neurons):
        self.Neurons = neurons 

    
    def create_random(self, amount, amount_wts, wt_range):
        self.Neurons = []
        for i in range(amount):
            wts = [random.randint(*wt_range) for i in range(amount_wts)]
            bias = random.randint(*wt_range)
            n = Neuron(wts, bias)
            self.Neurons.append(n)
    
    def get_outputs(self, inp):
        out = []
        for neuron in self.Neurons:
            output_sum = neuron.calculate_output(inp)
            out.append(output_sum)
        return out
    
    def mutate(self, wt_chance):
        for i, neuron in enumerate(self.Neurons):
            for j, wt in enumerate(neuron.Weights):
                if random.random() <= wt_chance:
                    self.Neurons[i].Weights[j] += random.uniform(-1.5, 1.5)

            if random.random() <= wt_chance:
                self.Neurons[i].Bias += random.uniform(-1.5, 1.5)
    
    def display(self):
        string = ""
        for neuron in self.Neurons:
            string += "[{}, {}], ".format(neuron.Weights, neuron.Bias)
        
        print(f"[{string}], ")
    
    
            
class Neuron:
    def __init__(self, weights, bias):
        self.Weights = weights
        self.Bias = bias
    
    
    
    def calculate_output(self, inp):
        if len(inp) != len(self.Weights):
            print("ERROR: Number of inputs and weights do not match")
            return
        
        out = 0
        for val, weight in zip(inp, self.Weights):
            out += val * weight
        return self.activation(out + self.Bias)
    
    def activation(self, out):
        return max(0, out)

 # Create Game! 
# map(tile or no tile) all depends on raycasting method

class Ray:
    def __init__(self, start_pos, angle, rects=[]):
        self.Angle = angle
        self.StartPos = start_pos
        
        self.update_ray(rects)
    
    def update_ray(self, rects):
        dx, dy = math.cos(self.Angle) * 10, math.sin(self.Angle) * 10
        curr_x, curr_y = self.StartPos
        collided = False
        while 0 < curr_x < WIDTH and 0 < curr_y < HEIGHT:
            for rect in rects:
                if rect.collidepoint((curr_x, curr_y)):
                    collided = True
            if collided:
                break
            else:
                curr_x += dx
                curr_y += dy

        self.End = [curr_x, curr_y]
    
    def get_length(self):
        return math.sqrt((self.End[0] - self.StartPos[0])**2 + (self.End[1] - self.StartPos[1])**2)
    
    def draw(self, surf):
        pygame.draw.line(surf, (150, 0, 200), self.StartPos, self.End, width=2)


# Car object
# moves forward, turn angle
class Car:
    def __init__(self, pos, max_vel, angular_vel, angle):
        self.pos = pos
        self.VEL = max_vel
        self.TURN_SPEED = angular_vel
        self.angle = angle
        self.color = (0, 255, 0)
        self.hit_checkpoints = []
    
    
    def move_forward(self):
        radians = math.radians(self.angle)
        dx, dy = math.cos(radians) * self.VEL, math.sin(radians) * self.VEL
        self.pos[0] += dx
        self.pos[1] += dy
    
    def turn_left(self):
        self.angle -= self.TURN_SPEED
    
    def turn_right(self):
        self.angle += self.TURN_SPEED
    
    def get_points(self):
        tri_h = 20
        tri_w = 5
        radians = math.radians(self.angle)
        points = [
            (self.pos[0] + math.cos(radians) * tri_h, self.pos[1] + math.sin(radians) * tri_h),
            (self.pos[0] + math.cos(radians + math.pi/2) * tri_w, self.pos[1] + math.sin(radians + math.pi/2) * tri_w),
            (self.pos[0] + math.cos(radians - math.pi/2) * tri_w, self.pos[1] + math.sin(radians - math.pi/2) * tri_w)
                 ]
        return points
    
    def draw(self, surf):
        points = self.get_points()
        pygame.draw.polygon(surf, self.color, points)


def create_rays(num_rays, angle, pos, fov, rects):
    curr_angle = angle - fov/2 if num_rays-1 > 0 else angle
    diff = fov/(num_rays-1) if num_rays-1 > 0 else 0
    rays = []
    for i in range(num_rays):
        radians = math.radians(curr_angle)
        ray = Ray(pos, radians, rects=rects)
        curr_angle += diff
        rays.append(ray)
    return rays


class TestCars:
    
    def test1(self):
        objects = [pygame.Rect(14, 35, 18, 653),
        pygame.Rect(18, 644, 562, 41),
        pygame.Rect(562, 20, 16, 657),
        pygame.Rect(119, 539, 451, 47),
        pygame.Rect(26, 409, 435, 45),
        pygame.Rect(127, 272, 435, 53),
        pygame.Rect(28, 168, 311, 24),
        pygame.Rect(201, 190, 55, 30),
        pygame.Rect(410, 150, 152, 30),
        pygame.Rect(410, 177, 34, 102),
        pygame.Rect(388, 33, 32, 111),
        pygame.Rect(138, 42, 253, 50),
        pygame.Rect(202, 84, 42, 34),]

        checkpoints = [pygame.Rect(399, 610, 46, 18),
        pygame.Rect(34, 543, 86, 35),
        pygame.Rect(169, 586, 30, 59),
        pygame.Rect(194, 453, 54, 87),
        pygame.Rect(458, 422, 53, 35),
        pygame.Rect(74, 281, 49, 25),
        pygame.Rect(224, 218, 27, 57),
        pygame.Rect(335, 162, 89, 18),
        pygame.Rect(306, 95, 22, 77),
        pygame.Rect(33, 68, 105, 19),
        pygame.Rect(373, 448, 40, 97),]
        car_presets = Car([520, 610], 10, 10, 180)
        # best:[[[[-1.1512033670954578, 3], -1.885351261772409], [[0.9733181288628376, -2.1602564861111633], 1.0551781363714483], [[-3, 1.9071957854788935], -0.2987994454596632], ], [[[1.0049117339874079, -1.2531857458744828, -1.7936881199394954], -5.575566439294432], [[-1.5977752122920235, 1, 2], -3], [[2, 2.5861927230498423, -2.0856621980147367], 1.8615282919510847], ]]
        
        start_net = None
        self.car_test(objects, checkpoints, car_presets, set_start=start_net)
    
    def car_test(self, object_list, checkpoint_list, car_obj, gen_time=5, pool_size=50, set_start=None):
        def fnCreate():
            hidden1 = Layer(None)
            hidden1.create_random(3, 2, [-3, 3])
            output = Layer(None)
            output.create_random(3, 3, [-3, 3])
            
            network = [hidden1, output]
            return [Chromosome(network, "Create"), deepcopy(car_obj)]

        def fnSetStartChrom(set_parent):
            net = []
            for i, layer in enumerate(set_parent):
                layer_obj = Layer([])
                for j, neuron in enumerate(layer):
                    wts, bias = neuron
                    neuron_obj = Neuron(wts, bias)
                    layer_obj.Neurons.append(neuron_obj)
                net.append(layer_obj)
            return Chromosome(net, "Create")
        
        def fnMutate(parent):
            child_genes = deepcopy(parent.Genes)
            for i, layer in enumerate(child_genes):
                child_genes[i].mutate(0.1)
            return [Chromosome(child_genes, "Mutate"), deepcopy(car_obj)]
        
        def fnCrossover(p1, p2):
            child_genes = crossover(deepcopy(p1.Genes), deepcopy(p2.Genes))
            return [Chromosome(child_genes, "Crossover"), deepcopy(car_obj)]
        
        def fnDisplay(candidate):
            time_diff = str(datetime.datetime.now() - start_time)
            fit = candidate.Fitness
            for layer in candidate.Genes:
                layer.display()
            print(f"Fitness: {fit} | {time_diff}")
                
        
        
        
        start_time = datetime.datetime.now()
        gen_start = time.time()
        
        GEN = 1
        parents = []
        
        for i in range(pool_size):
            if set_start != None:
                parent_chrom = fnSetStartChrom(set_start)
                parent = fnMutate(parent_chrom)
            else:
                parent = fnCreate()
            parents.append(parent)
        two_best = sorted(parents[:2], key=lambda parent:parent[0].Fitness)
        
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    for best in two_best:
                        fnDisplay(best[0])
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        print(pygame.mouse.get_pos())
                    
            DISPLAYSURF.fill((0, 0, 0))
            for i, obj in enumerate(object_list):
                pygame.draw.rect(DISPLAYSURF, (255, 0, 0), obj, width=5)
            for i, checkpoint in enumerate(checkpoint_list):
                pygame.draw.rect(DISPLAYSURF, (0, 255, 0), checkpoint, width=5)
            
            if time.time() - gen_start >= gen_time or len(parents) == 0:
                print()
                print(f"GENERATION {GEN} OVER! | {time.time() - gen_start}")
                print()
                for best in two_best:
                    fnDisplay(best[0])
                    print()
                gen_start = time.time()
                GEN += 1
                if GEN % 5 == 0:
                    gen_time += 2
                
                parents = []
                for i in range(pool_size):
                    funcs = [lambda p1, p2: fnMutate(p1), lambda p1, p2: fnCrossover(p1, p2)]
                    func = random.choice(funcs)
                    p1, p2 = random.sample(two_best, 2)
                    child = func(p1[0], p2[0])
                    parents.append(child)
            
            # update parents
            for i, parent in enumerate(parents):
                chrom, car = parent[0], parent[1]
                rays = create_rays(3, car.angle, car.pos, 180, object_list)
                #for ray in rays:
                 #   ray.draw(DISPLAYSURF)
                inp = [ray.get_length() for ray in rays]
                left_rights = inp[0] - inp[2]
                forward = inp[1]
                
                out = feedforward(chrom.Genes, [left_rights, forward]) #[left-right space, forward space]
                if out[0] > out[1]:
                    car.turn_left()
                    chrom.Fitness -= 0.01
                elif out[1] > out[0]:
                    car.turn_right()
                    chrom.Fitness -= 0.01
                if out[2] > 0:
                    car.move_forward()
                car.draw(DISPLAYSURF)
                
                points = car.get_points()
                add = False
                for checkpoint in checkpoint_list:
                    for point in points:
                        if checkpoint.collidepoint(point) and checkpoint.topleft not in car.hit_checkpoints:
                            add = True
                            car.hit_checkpoints.append(checkpoint.topleft)
                            break
                    if add:
                        chrom.Fitness += 50 
                        break
                        
                remove = False
                for obj in object_list:
                    for point in points:
                        if obj.collidepoint(point):
                            remove = True
                            break
                if not remove:
                    parents[i] = [chrom, car]
                    if chrom.Fitness > two_best[0][0].Fitness and chrom.Genes != two_best[0][0].Genes:
                        two_best[0] = [deepcopy(chrom), car]
                        two_best.sort(key=lambda parent: parent[0].Fitness)
                        parents[i][1].color = (0, 0, 255)
                    else:
                        parents[i][1].color = (0, 255, 0)
                else:
                    parents.pop(i)
                
            
            pygame.display.update()
            clock.tick(30)
        
        



TestCars().test1()

In [ ]:
""

In [ ]:
""" Create Map Maker for car game"""
'''

list for objects

click two points too add object
store first loc in click1
on second click:
    create rect with two points

'''
import pygame, sys


pygame.init()
DISPLAYSURF = pygame.display.set_mode((600, 700))
clock = pygame.time.Clock()

click1 = None
append_type = 1
remove = False

objects = [
pygame.Rect(14, 35, 18, 653),
pygame.Rect(18, 644, 562, 41),
pygame.Rect(562, 20, 16, 657),
pygame.Rect(119, 539, 451, 47),
pygame.Rect(26, 409, 435, 45),
pygame.Rect(127, 272, 435, 53),
pygame.Rect(28, 168, 311, 24),
pygame.Rect(201, 190, 55, 30),
pygame.Rect(410, 150, 152, 30),
pygame.Rect(410, 177, 34, 102),
pygame.Rect(388, 33, 32, 111),
pygame.Rect(138, 42, 253, 50),
pygame.Rect(186, 96, 65, 22),]
checkpoints = [pygame.Rect(424, 603, 20, 20),
pygame.Rect(279, 602, 20, 20),
pygame.Rect(68, 594, 20, 20),
pygame.Rect(326, 489, 20, 20),
pygame.Rect(496, 407, 20, 20),
pygame.Rect(303, 360, 20, 20),
pygame.Rect(172, 355, 20, 20),
pygame.Rect(70, 254, 20, 20),
pygame.Rect(218, 234, 20, 20),
pygame.Rect(368, 171, 20, 20),
pygame.Rect(213, 127, 20, 20),
pygame.Rect(200, 484, 20, 20),
pygame.Rect(68, 498, 20, 20),]

while True:
    remove = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            for obj in objects:
                print(f"pygame.Rect({obj.x}, {obj.y}, {obj.width}, {obj.height}),")
            print()
            print("Checkpoints")
            print()
            for obj in checkpoints:
                print(f"pygame.Rect({obj.x}, {obj.y}, {obj.width}, {obj.height}),")
            pygame.quit()
            sys.exit()
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                click1 = list(event.pos)
            elif event.button == 3:
                remove = True
        if event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1:
                width, height = abs(click1[0] - event.pos[0]), abs(click1[1] - event.pos[1]) 
                x, y = min(click1[0], event.pos[0]), min(click1[1], event.pos[1])
                rect = pygame.Rect(x, y, width, height)
                if append_type == 1:
                    objects.append(rect)
                elif append_type == 2:
                    checkpoints.append(rect)
        
                click1 = None
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_BACKSPACE:
                if append_type == 1:
                    if len(objects) >= 1:
                        del objects[-1]
                if append_type == 2:
                    if len(checkpoints) >= 1:
                        del checkpoints[-1]
                
            if event.key == pygame.K_1:
                append_type = 1
            if event.key == pygame.K_2:
                append_type = 2
    
    DISPLAYSURF.fill((0, 0, 0))
    
    if click1 != None:
        mx, my = pygame.mouse.get_pos()
        width, height = abs(click1[0] - mx), abs(click1[1] - my) 
        x, y = min(click1[0], mx), min(click1[1], my)
        rect = pygame.Rect(x, y, width, height)
        pygame.draw.rect(DISPLAYSURF, (255, 255, 0), rect, width=5)
    
    for i, obj in enumerate(objects):
        pygame.draw.rect(DISPLAYSURF, (255, 0, 0), obj, width=5)
        if remove:
            mx, my = pygame.mouse.get_pos()
            if obj.collidepoint((mx, my)):
                objects.pop(i)

    for i, prize in enumerate(checkpoints):
        pygame.draw.rect(DISPLAYSURF, (255, 255, 0), prize, width=5)
        if remove:
            mx, my = pygame.mouse.get_pos()
            if prize.collidepoint((mx, my)):
                checkpoints.pop(i)
    
    pygame.display.update()
    clock.tick(30)